In [1]:
import numpy as np
import math
import vectorbt as vbt
import pandas_ta as ta
import talib as talib
import pandas as pd
from hyperopt import fmin, tpe, hp
from hyperopt.pyll import scope
from datetime import datetime
from pdta_vt_utils import dl, dtmask

# Portfolio Setting

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings

Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Data Collection

In [3]:
asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]

print("="*100)
print("Tickers by index #")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")

asset_name = asset_tickers[3] # Change index for different symbol
print("="*100)
print(f"Selected Benchmark | Asset: {asset_name}")
print("="*100)

assets = dl(asset_tickers, lc_cols=True)

start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')

print("="*100)
print("Available Data:")
print("="*100)
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)

assetdf = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
assetdf.name = asset_name
print("="*100)
print(f"Analysis of:  {assetdf.name}{crs}")
print("="*100)

Tickers by index #
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL, 4: ^GSPC, 5: SPY
Selected Benchmark | Asset: SPXL
[i] Downloading: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
[+] AAPL(10632, 7) Monday February 13, 2023, NYSE: 15:02:41
[+] TSLA(3179, 7) Monday February 13, 2023, NYSE: 15:02:42
[+] TWTR(2259, 7) Monday February 13, 2023, NYSE: 15:02:42
[+] SPXL(3592, 8) Monday February 13, 2023, NYSE: 15:02:42
[+] ^GSPC(23894, 7) Monday February 13, 2023, NYSE: 15:02:43
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 15:02:43
[*] Download Complete

Available Data:
Assets: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
Analysis of:  SPXL from 2010-01-01 00:00:00-05:00 to 2015-01-01 00:00:00-05:00


In [4]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,4.100748,4.207281,4.099988,4.200433,28238400,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,4.195866,4.242284,4.147166,4.240001,33206400,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,4.227826,4.278810,4.217934,4.251415,44194800,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,4.229349,4.315335,4.188257,4.301638,43773600,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,4.264353,4.352623,4.240002,4.347295,39685200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372379,21.452074,21.266901,21.290340,1291600,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,21.445042,21.606777,21.421602,21.480202,2223200,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,21.400506,21.634904,21.391130,21.559896,2677600,0.0,0.0,0.0


# Indicator Setting Use this section for custome indicators

In [5]:
def stochastic(high, low, close, lookback, k, d):
    """function to calculate Stochastic Oscillator
    lookback = lookback period
    k and d = moving average window for %K and %D"""
    df = pd.DataFrame({'high':high, 'low':low, 'close':close})
    HH = df["high"].rolling(lookback).max()
    LL = df["low"].rolling(lookback).min()
    K = (100 * (df["close"] - LL)/(HH - LL)).rolling(k).mean()
    D = K.rolling(d).mean()
    return K, D

def stochastic_momentum(high, low, close, win_h, win_l):
    h = pd.DataFrame(high)
    l = pd.DataFrame(low)
    c = pd.DataFrame(close)
    high14 = h.rolling(win_h).max()
    low14 = l.rolling(win_l).min()
    perc_k = (c - low14) * 100 / (high14 - low14)
    perc_d = perc_k.rolling(3).mean()
    return perc_k, perc_d


Stochastic_momentum = vbt.IndicatorFactory(
    class_name='Stochastic_momentum',
    input_names=['high', 'low', 'close'],
    param_names=['lookback', 'k', 'd'],
    output_names=['perc_k', 'per_d'], ).from_apply_func(stochastic)




# Optimization

In [6]:
below = 30
above = 70


def optimize(params):
    k_window, d_window, d_ewm, sl_stop = params
    sthoc = vbt.STOCH.run(assetdf.High, assetdf.Low, assetdf.Close, k_window=5, d_window=3, d_ewm=3)


    entries_st = (sthoc.percent_k_below(30) & sthoc.percent_d_below(30)) & sthoc.percent_d_crossed_below(sthoc.percent_k)
    exits_st =   (sthoc.percent_k_above(70) & sthoc.percent_d_above(70)) & sthoc.percent_d_crossed_above(sthoc.percent_k)

    pf = vbt.Portfolio.from_signals(close=assetdf['Close'], entries=entries_st, exits=exits_st, sl_stop=sl_stop, direction="longonly")

    print(f"total profit {pf.total_profit()} with parameters: {k_window}, {d_window}, {d_ewm}, {sl_stop}")
    return np.array([pf.total_profit()])


def optimize_all(params):
    res = optimize(params)
    return sum(res) * -1




best = fmin(optimize_all,
            space=[scope.int(hp.uniform('k_window', 1, 8)),
                   scope.int(hp.uniform('d_window', 1, 6)),
                   scope.int(hp.uniform('d_ewm', 1, 5)),
                   hp.uniform('sl_stop', 0.01, 0.1)],
            algo=tpe.suggest,
            max_evals=100)

print(best)

total profit 288.5081035469209 with parameters: 6, 3, 4, 0.07834365415151166
total profit 376.770806396667 with parameters: 2, 4, 3, 0.03057867195769038      
total profit 456.6133388244567 with parameters: 4, 2, 1, 0.028416340660087566    
total profit 479.15666511280483 with parameters: 1, 3, 3, 0.037725106760498395   
total profit 242.934262417225 with parameters: 3, 4, 2, 0.08384675595462807       
total profit 403.3856887498391 with parameters: 2, 3, 1, 0.02286756369114922      
total profit 379.3210699794348 with parameters: 7, 4, 2, 0.05672292632509645      
total profit 335.1273127381162 with parameters: 7, 1, 4, 0.03166231335920371      
total profit 292.45116807563534 with parameters: 6, 3, 4, 0.07590383430835036     
total profit 224.5745885714383 with parameters: 4, 1, 2, 0.09365153494079309      
total profit 379.3210699794348 with parameters: 7, 3, 3, 0.05584082190194275       
total profit 485.4018164359775 with parameters: 7, 3, 1, 0.03630823030944074       
total profi

In [11]:
k_window = 8 #math.ceil(best['k_window'])
d_window = 3 #math.ceil(best['d_window'])
d_ewm = 3 #math.ceil(best['d_ewm'])
sl_stop = best['sl_stop']

In [12]:
sthoc = vbt.STOCH.run(assetdf.High, assetdf.Low, assetdf.Close, k_window=k_window, d_window=d_window, d_ewm=d_ewm)
entries_st = (sthoc.percent_k_below(below) & sthoc.percent_d_below(below)) & sthoc.percent_d_crossed_below(sthoc.percent_k)
exits_st =   (sthoc.percent_k_above(above) & sthoc.percent_d_above(above)) & sthoc.percent_d_crossed_above(sthoc.percent_k)
pf_st = vbt.Portfolio.from_signals(assetdf.Close, entries=entries_st, exits=exits_st, sl_stop=sl_stop)
pf_st.stats()

Start                         2010-01-04 00:00:00-05:00
End                           2014-12-31 00:00:00-05:00
Period                               1258 days 00:00:00
Start Value                                      1000.0
End Value                                   1293.354755
Total Return [%]                              29.335476
Benchmark Return [%]                         389.505827
Max Gross Exposure [%]                            100.0
Total Fees Paid                              176.612978
Max Drawdown [%]                              50.409323
Max Drawdown Duration                 866 days 00:00:00
Total Trades                                         34
Total Closed Trades                                  34
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       50.0
Best Trade [%]                                25.281987
Worst Trade [%]                              -20

In [13]:
pf_st.plot()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f97dc351-81b2-4628-9e99-a232caef7934',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 4.20043278,  4.24000072,  4.25141525, ..., 21.55989647, 21.2129879 ,
                          20.56136322]),
              'yaxis': 'y'},
             {'customdata': array([[  0.        , 260.63475344,   2.49376559],
                                   [  2.        , 256.35709971,   2.38732437],
                                   [  4.        , 239.88816577,   2.19819883],
                                   [  6.        , 274.46445101,   2.07859466],
                                   [  8.        , 319.67488686,   2.60279419],
                                   [ 10.        , 197.237401  ,   2.85153355],
                                   [ 12.        , 189.9264811 ,   3.0824192 ],
                                   [ 14.        , 194.02059631,   2.9231862 ],
                                   [ 16.        , 192.38874338,   2.77879802],
                                   [ 18.        , 203.39031252,   3.25393386],
                                   [ 20.        , 242.4924576 ,   3.36429851],
                                   [ 22.        , 244.07567447,   2.8466603 ],
                                   [ 24.        , 211.89912351,   2.26847816],
                                   [ 26.        , 184.40683778,   1.94924418],
                                   [ 28.        , 163.29786142,   1.77376078],
                                   [ 30.        , 124.44568246,   1.92611931],
                                   [ 32.        , 132.62422677,   2.02855127],
                                   [ 34.        , 150.0595781 ,   1.91029433],
                                   [ 36.        , 131.90024334,   2.20111878],
                                   [ 38.        , 141.19279061,   2.09546702],
                                   [ 40.        , 113.87370215,   2.26376575],
                                   [ 42.        , 112.76052499,   2.50263021],
                                   [ 44.        , 100.34386764,   2.61610893],
                                   [ 46.        , 103.70290931,   2.45147031],
                                   [ 48.        , 102.37747805,   2.73468367],
                                   [ 50.        , 100.0739436 ,   2.60084057],
                                   [ 52.        , 104.16080515,   2.80312355],
                                   [ 54.        ,  87.58093512,   2.97999668],
                                   [ 56.        ,  85.14723963,   2.7121127 ],
                                   [ 58.        ,  84.06762795,   3.04565277],
                                   [ 60.        ,  75.70276446,   3.26960169],
                                   [ 62.        ,  66.99326445,   3.10084381],
                                   [ 64.        ,  66.48396447,   2.91088968],
                                   [ 66.        ,  70.92304548,   2.71276805]]),
              'hovertemplate': (

In [14]:
sthoc.plot()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'name': '%K',
              'showlegend': True,
              'type': 'scatter',
              'uid': '3c36ccb0-383d-46df-bf59-b4bf019d1d37',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([        nan,         nan,         nan, ..., 97.78086302, 76.65370625,
                           1.71677818])},
             {'name': '%D',
              'showlegend': True,
              'type': 'scatter',
              'uid': '367978fa-79f5-46ea-99a5-342615211c4f',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'y': array([        nan,         nan,         nan, ..., 96.28259994, 86.46815309,
                          44.09246564])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'shapes': [{'fillcolor': 'purple',
                           'layer': 'below',
                           'line': {'width': 0},
                           'opacity': 0.2,
                           'type': 'rect',
                           'x0': Timestamp('2010-01-04 00:00:00-0500', tz='America/New_York'),
                           'x1': Timestamp('2014-12-31 00:00:00-0500', tz='America/New_York'),
                           'xref': 'x',
                           'y0': 30,
                           'y1': 70,
                           'yref': 'y'}],
               'template': '...',
               'width': 700,
               'yaxis': {'range': [-5, 105]}}
})